**Exercício 03: Redes Neurais Artificiais CLASSIFICAÇÃO**
Célula de notas.

**Importando bibliotecas**

In [1]:
import pandas as pd
import numpy as np

**Obtendo dados**

In [2]:
# Célula de código do python
try:
    print('Obtendo dados...')

    # Constante dos dados
    ENDEREÇO_DADOS = 'https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv'

    # Obtendo dados e delimitando
    df = pd.read_csv(ENDEREÇO_DADOS, sep=',', encoding='utf-8')

    # Filtro por estado
    df = df[df['reviewer_state'] =='RJ'][['review_text','overall_rating']]

    # Excluindo dados que não existem (NaN) # equivalente a dados nulos null
    df = df.dropna(subset=['review_text','overall_rating'])
    
    

    # Transformando colunas em arrays
    texts = np.array(df['review_text'])
    rating = np.array(df['overall_rating'])

    print(df['overall_rating'].unique()) # exibir categorias
    print(df['overall_rating'].value_counts()) # exibir quantidade de valores por categoria

    print(df.head())
except Exception as e:
    print('Erro ao obter dados: ',e)

Obtendo dados...
[4 1 5 2 3]
overall_rating
5    6122
4    3933
1    3856
3    2127
2    1029
Name: count, dtype: int64
                                          review_text  overall_rating
0   Estou contente com a compra entrega rápida o ú...               4
9   MEU PRODUTO NAO FOI ENTREGUE E A AMERICANAS ES...               1
11  Produto maravilhoso! Não é barulhento, fácil m...               5
15  a mochila nao esta fechando direito por isso n...               2
22  Melhor do que imaginava. Superou minhas expect...               5


C:\Users\36131872024.1\AppData\Local\Temp\ipykernel_29832\3177015159.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ENDEREÇO_DADOS, sep=',', encoding='utf-8')


**Vetorização**

Vetorizar o texto: transformar em números.
Os números do rating, já estão vetorizados pois já estão transformados em array.

In [3]:
# Bibioteca para trabalhar com redes neurais artificiais
# Tersorflow: https://www.tensorflow.org/?hl=pt-br
from tensorflow.keras.preprocessing.text import Tokenizer # type:ignore  # Tokenizar
from tensorflow.keras.preprocessing.sequence import pad_sequences # type:ignore  # Ajustar tamanho do vetor
try:
    print('Vetorizando texto...')

    # Passo 1: tokenizar
    tokenizer = Tokenizer() 
    # transforma todas as palavras do texto em tokens e cria um dicionário com índices das palavras
    # Transforma texto em números

    # Passo 2: Criar o dicionário
    # fit_on_texts: Cria o vocabulário, através do dicionário
    # associando cada token a um índice numérico a cada palavra
    # lembrando que se a palavra aparecer mais de uma vez, ela vai receber o mesmo índice numérico.
    tokenizer.fit_on_texts(texts)

    # Passo 3: Vetorizar, ou seja, transformar os tokens em números,
    # a partir do dicionário criado no passo 2
    vetores = tokenizer.texts_to_sequences(texts)

    # Passo 4: Padronização do tamanho do vetor - pad (adicina zeros 0)
    padded_vetores = pad_sequences(vetores)

    print(padded_vetores) # Vetorzão (Aula teórica)
    print('Textos vetorizados com sucesso!')

except Exception as e:
    print('Erro ao vetorizar texto: ',e)

Vetorizando texto...
[[    0     0     0 ...   180    10   825]
 [    0     0     0 ...    11    36   329]
 [    0     0     0 ...    18    10  1205]
 ...
 [    0     0     0 ... 17732  9137 17733]
 [    0     0     0 ...     2   103  9137]
 [    0     0     0 ...    20    43   558]]
Textos vetorizados com sucesso!


**Rede Neural**

In [5]:
from tensorflow.keras.models import Sequential # type:ignore # Definir o modelo de rede neural utilizada
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout # type:ignore  # Camadas da rede neural
# Embedding: Classe do tensorflow

from tensorflow.keras.optimizers import Adam # type:ignore # otimizador de taxa de aprendizado
try:
    print('Contruindo a rede neural...')

    # Constantes do modelo:

    # 1ª Constante: Tamanho do vocabulário (tamanho do dicionário)
    VOCAB_SIZE = len(tokenizer.word_index)+1 # +1 pois começa em 0

    # 2ª Constante: Tamanho máximo da sequência
    # É o comprimento máximo de um texto
    MAX_SEQUENCE_LENGHT = padded_vetores.shape[1] # shape: tamanho das colunas dentro do vetor
    # [   0    0    0 ...    0    0    3]

    # 3ª Constante: Tamanho do vetor de entrada
    # A literatura recomenda que inicia-se por uma quantidade igual a raiz quadrada do tamanho do vocabulário
    # Se o volume de dados for de larga escala, pode-se testar iniciando com um tamanho maior
    # Se o volume de dados for muito pequeno, pode-se testar iniciando com um tamanho menor
    # Cuidado com o overfitting, quando o modelo aprende de mais e começa a perder a 
    # capacidade de generalizar melhor, ou seja, observar todas as diferenças textuais.
    # Overfitting pode ser observado no treino da rede neural.
    VETOR_LENGHT = int(np.sqrt(VOCAB_SIZE)) # raiz do tamanho do vocabuário

    # Construção da rede neural (de regressão). Modelo sequencial
    # Sequential é um fluxo linear de camadas (conforme visto na Aula 02_RNA.pptx)
    # São processadas em ordem
    model = Sequential()

    # Camada de entrada
    # Embeddings, na qual os vetores de texto são inseridos
    model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=VETOR_LENGHT, input_length=MAX_SEQUENCE_LENGHT))

    # Camada oculta ou intermediária
    # LSTM - Long short-term memory, "memória de curto e longo prazo"
    # É onde o modelo treina, baseado nos seus vetores
    # Números de unidades de memória, que basicamente á quantidade de "neurônios"
    # Quanto maior a quantidade de neurônios, maior a acurácia, maior o custo de processamento
    # Só é necessário ter uma camada para ter uma saída. Podem ter mais camadas.
    # Primeiro TESTE, somente com 1 camada. Cuidado com o overfitting.
    
   
    # Primeira camada oculta
    model.add(LSTM(128, return_sequences=True)) # Uma camada intermediária terá 128 neurônios
    # Para adicionar mais camadas, basta repetir o comando model.add(LSTM(128))
    # É recomendado adicionar apenas uma camada de início. 
    # Após rodar o código, se necessário, vai adicionando novas camadas.
    # LSTM é um tipo de camada oculta. Existem outras.
    model.add(LSTM(64))
    
    # Próximas camadas ocultas...
    #model.add(LSTM(128))

    # Camada de Saída - Camada Densa
    # Modelo de Classificação: Precisa ajustar para a quantidade de classes (categorias)
    # 5 Camadas. É esse o modelo do código exemplo03.py
    # Categorias: 1, 2, 3, 4 e 5
    # Modelo de Classificação: tem mais camadas de saída
    model.add(Dense(5, activation='softmax')) # ****** AJUSTE DE MODELO ******
    model.add(Dropout(0.05))

    # Construir o modelo
    # É literalmente pegar as definições anteriores e construir o modelo
    # input_shape: é o formato de dados de entrada e ainda o tamanho máximo do texto(MAX_SEQUENCE_LENGHT)
    # None: o modelo verifica o dado de entrada e define o formato
    model.build(input_shape=(None,MAX_SEQUENCE_LENGHT))

    # Otimizador de taxa de aprendizado
    # importante para ajustar em casos de overfitting
    # Adam: É o otimizador que vai ajustar essa taxa de aprendizado
    # learning_rate: Quanto menor, melhor o aprendizado. Menos risco de overfitting.
    otimizador = Adam(learning_rate=0.0001) # Reduzindo para compensar o desbalanceamento das classes nos dados de entrada

    # Compilar o modelo
    # Verificar se possui algum erro
    # Além disso vamos informar o otimizador e a nossa métrica de perda (loss)
    # Usada para avaliar o modelo. Ver se existe overfitting ou não
    # loss - sparse_categorical_crossentropy
    model.compile(optimizer=otimizador, loss='sparse_categorical_crossentropy')

    model.summary()
    print('Modelo configurado e criado com sucesso!')



except Exception as e:
    print('Erro ao construir a rede neural: ',e)

Contruindo a rede neural...


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 529, 133)       │     2,358,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 529, 128)       │       134,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           325 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 5)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,542,765 (9.70 MB)

 Trainable params: 2,542,765 (9.70 MB)

 Non-trainable params: 0 (0.00 B)

Modelo configurado e criado com sucesso!


In [6]:
from sklearn.model_selection import train_test_split
# Treinar o modelo
try:
    print('Treinar o modelo de rede neural')

    X_train, X_test, y_train, y_test = train_test_split(
        padded_vetores,
        rating,
        test_size=0.2,
        random_state=42
    )
    
    # Para RNA de Classificação:
    # Quando construimos uma rede neural de classificação, os rótulos que a RNA atribui, não começa em 1, começa em zero 0.
    # Sendo que os dados são categorizados em 1, 2, 3, 4 e 5. 
    # Necessário ajustar as saídas (os y's)
    y_train_adjusted = y_train -1
    y_test_adjusted = y_test -1

    pesos = len(df['overall_rating']) / df['overall_rating'].value_counts() # ***** AJUSTE DE MODELO*****
    print(pesos)

    # o treino da rede neural
    model.fit(
        X_train, 
        y_train_adjusted,
        epochs=5, # Épocas de estudo da rede neural. Inicia com 5 e vai avaliando. 
        # Quanto mais epochs, mais aprendizado (mais processamento), porém cuidado com o overfitting.
        # A regra é a experimentação, avaliando a perda de treino com a perda de teste.
        # A busca é: loss (perda de treino) menor que val_loss (perda de teste).
        batch_size=32, # Bloco de dados de treino. # Quando maior o tamanho da batch, tende menor o aprendizado, porém evita overfitting
        # Quanto menor o tamanho da batch, tende a um maior aprendizado

        class_weight = pesos.to_dict(), # ***** AJUSTE DE MODELO*****
        validation_data=(X_test, y_test_adjusted)
    )
    # No caso da classificação, o loss precisa diminuir para não indicar overfitting

except Exception as e:
    print('Erro ao treinar o modelo de rede neural: ',e)

Treinar o modelo de rede neural
overall_rating
5     2.787814
4     4.339436
1     4.426089
3     8.023977
2    16.586006
Name: count, dtype: float64
Epoch 1/5
427/427 ━━━━━━━━━━━━━━━━━━━━ 102s 235ms/step - loss: 12.3600 - val_loss: 1.6679
Epoch 2/5
427/427 ━━━━━━━━━━━━━━━━━━━━ 106s 248ms/step - loss: 11.7793 - val_loss: 1.4943
Epoch 3/5
427/427 ━━━━━━━━━━━━━━━━━━━━ 107s 250ms/step - loss: 10.7904 - val_loss: 1.2682
Epoch 4/5
427/427 ━━━━━━━━━━━━━━━━━━━━ 106s 249ms/step - loss: 9.7434 - val_loss: 1.2486
Epoch 5/5
427/427 ━━━━━━━━━━━━━━━━━━━━ 107s 251ms/step - loss: 9.4088 - val_loss: 1.2612


In [7]:
# Textos novos inseridos para realizar a previsão, baseada no modelo de rede neural treinado.
# Nesse momento, você pode conectar em uma rede de dados (instagram...), para realizar as previsões de sentimento.

novos_textos = ['Muito bom, gostei bastante. Top demais! Compensa muito! Exxcelente!!! Melhor produto da minha vida, não sei como vivi sem ele até hoje. Nada se compara com esse produto, muito bom',
                'Não recomendo, péssimo produto. Não funciona. Uma grande MERDA!!!! Nunca vi coisa tão ruim na minha vida!!! Palhaçada! Me arrependo profundamente de ter comprado!!!!']

# Vetorizar
novas_sequencias = tokenizer.texts_to_sequences(novos_textos)
novas_sequencias_padded = pad_sequences(novas_sequencias)

# Análise preditiva
predicoes = model.predict(novas_sequencias_padded)

print('Previsões: ', predicoes) # com a ativação do softmax, podemos retirar o +1

# Resultado em escala de 1 a 5. Sendo 1 ruim e 5 bom.
# A probabilidade apresentada não é percentual.

# Conferir código. Resultado não confere

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
Previsões:  [[0.00260395 0.00456954 0.05614957 0.40133768 0.5353393 ]
 [0.53987336 0.2614718  0.06625181 0.06333688 0.06906611]]
